In [56]:
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import os
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

In [57]:
def convert_df_to_numpy(df, scaler):
  y = df['label'].to_numpy().astype(np.int32)
  X = scaler.transform(df.drop(columns=['label']).to_numpy())
  return X, y

center_surround_train_df = pd.read_csv("./center_surround_train.csv")
center_surround_test_df = pd.read_csv("./center_surround_test.csv")
center_surround_valid_df = pd.read_csv("./center_surround_valid.csv")

center_surround_scaler = StandardScaler()
center_surround_scaler.fit(center_surround_train_df.drop(columns=['label']).to_numpy())

center_surround_train_data = convert_df_to_numpy(center_surround_train_df, center_surround_scaler)
center_surround_test_data = convert_df_to_numpy(center_surround_test_df, center_surround_scaler)
center_surround_valid_data = convert_df_to_numpy(center_surround_valid_df, center_surround_scaler)

print(center_surround_train_data[0][:5])
print(center_surround_train_data[1][:5])

[[-1.36846388  1.54362905]
 [ 0.13476095 -0.35808876]
 [-1.17397245  2.0546884 ]
 [ 0.12212715 -0.3837286 ]
 [-0.08095132 -1.53810042]]
[1 0 1 0 1]


In [58]:
INPUT_DIM = 2
OUTPUT_DIM = 2
HIDDEN_NODES = 4
LEARNING_RATE = 0.01

In [63]:
class FeedForwardNeuralNetwork:
  def __init__(self, train_data, test_data, valid_data, input_dim, output_dim, hidden_nodes, learning_rate, hidden_activation="sigmoid", output_activation='sigmoid'):
    self.train_data = train_data
    self.test_data = test_data
    self.valid_data = valid_data
    self.input_dim = input_dim
    self.output_dim = output_dim
    self.hidden_nodes = hidden_nodes
    self.learning_rate = learning_rate
    self.hidden_activation = self.sigmoid if hidden_activation == "sigmoid" else None 
    self.output_activation = self.sigmoid if output_activation == "sigmoid" else None 

    self.W_input_hidden = None
    self.W_hidden_output = None
    self.B_input_hidden = None
    self.B_hidden_output = None

    self.initialize_weights_and_biases()
    self.forward()
  
  def sigmoid(self, x):
    return 1/(1 + np.exp(-x))

  def initialize_weights_and_biases(self):
    """
    Initialize weights and biases for input-hidden and hidden-output layers.

    Shapes:
      Input -> Hidden:  (2, 4) weights, (4,) biases
      Hidden -> Output: (4, 2) weights, (2,) biases
    """
    print("**Entered the weight and biases initialization**")
    self.W_input_hidden = np.random.random((self.input_dim, self.hidden_nodes))
    self.B_input_hidden = np.zeros((self.hidden_nodes,))

    self.W_hidden_output = np.random.random((self.hidden_nodes, self.output_dim))
    self.B_hidden_output = np.zeros((self.output_dim))

    return

  def forward(self):
    """
    Perform forward pass through hidden and output layers with activations.

    Output (first 3 points):
      [[0.67646572 0.6672606 ]
      [0.64835416 0.62535679]
      [0.71371663 0.69567189]]
    """
    
    print("\n**Entered the forward pass**")

    # Input to hidden layer with activation 
    train_features = self.train_data[0]
    hidden_logits = np.dot(train_features, self.W_input_hidden) + self.B_input_hidden
    hidden_output = self.hidden_activation(hidden_logits)
    
    # The output from the first layer (hidden_output) is now the input into the next layer
    output_logits = np.dot(hidden_output, self.W_hidden_output) + self.B_hidden_output
    output = self.output_activation(output_logits)

    print(output[:3])
    return output
    

FeedForwardNeuralNetwork(
  train_data=center_surround_train_data,
  test_data=center_surround_test_data,
  valid_data=center_surround_valid_data,
  input_dim=INPUT_DIM,
  output_dim=OUTPUT_DIM,
  hidden_nodes=HIDDEN_NODES,
  learning_rate=LEARNING_RATE
)

**Entered the weight and biases initialization**

**Entered the forward pass**
[[0.76182775 0.71978382]
 [0.69546485 0.69318456]
 [0.78956595 0.74231014]]
